# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os, requests
from IPython.display import Markdown
from dotenv import load_dotenv
from openai import OpenAI



In [ ]:
# constants

MODEL_GPT = 'openai/gpt-5-nano'
MODEL_LLAMA = 'llama3.2:1b'

In [ ]:
# set up environment
load_dotenv(override=True)

openai_api_key = os.getenv('OPENROUTER_API_KEY')
if openai_api_key:
    print(f"Openrouter API Key exists and begins {openai_api_key[:8]}")
else:
    print("Openrouter API Key not set")



In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": question}
    ]

In [ ]:
# Get gpt-5-mini to answer, with streaming

openai = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openai_api_key,
)

response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)
result = response.choices[0].message.content
display(Markdown(result))

In [ ]:
# Check if Ollama is running
requests.get("http://localhost:11434").content

In [ ]:
# Get Llama 3.2 to answer

openai = OpenAI(
  base_url="http://localhost:11434/v1",
  api_key='ollama',
)

response = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages)
result = response.choices[0].message.content
display(Markdown(result))